<a href="https://colab.research.google.com/github/sarthag/MNIST-Dataset/blob/main/MNIST_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import torchvision.transforms as transforms
from torch.utils.data.dataset import TensorDataset
import time

In [2]:
#define the NN model
class NN(nn.Module):
  def __init__(self, input_size, num_classes):
    super(NN, self).__init__()
    self.fc1 = nn.Linear(input_size, 50)
    self.fc2 = nn.Linear(50, num_classes)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

In [4]:
#Basic Test
model = NN(784, 10)
x = torch.randn(64,784)
model = model.to(device)
x = x.to(device)
start = time.process_time()
print(model(x).shape)
end = time.process_time()
print("time: ", end - start)

torch.Size([64, 10])
time:  0.0009015619999996893


In [5]:
#parameters
in_channel = 1
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 100

In [6]:
#loading the data

X,y = fetch_openml("mnist_784", version = 1, return_X_y = True)

X = X.astype(np.float32)
y = np.int_(y)
X = X.reshape(X.shape[0], 784)
print(X.shape, y.shape)

(70000, 784) (70000,)


In [7]:
X_tensor = torch.from_numpy(X)
y_tensor = torch.from_numpy(y)
y_tensor = y_tensor.type(torch.LongTensor)
X_train, X_test, y_train, y_test = train_test_split(X_tensor,y_tensor, test_size = (1/7), random_state = 42)

In [8]:
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle = True)

In [9]:
#initialise network
model = NN(784, num_classes)
loss_fun = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate) 

In [10]:
def chk_accuracy(loader, model):
    
  num_correct = 0
  num_samples = 0
  model.eval()
    
  with torch.no_grad():
    for x, y in loader:
      x = x.to(device = device)
      y = y.to(device = device)
      scores = model(x)
      predictions = scores.argmax(1)
      num_correct += sum((predictions == y))
      num_samples += predictions.size(0)
            
    return float(num_correct)/float(num_samples)

In [11]:
#Train the network
for epoch in range(num_epochs):
    model.train()
    if torch.cuda.is_available(): torch.cuda.empty_cache()
    model = model.to(device = device)

    loss_train = 0
    start = time.process_time()
    for batch, (data, targets) in enumerate(train_loader):
      data = data.to(device = device)
      targets = targets.to(device= device)
        
      #Forward Prop
      scores = model(data)
      loss = loss_fun(scores, targets)
        
      #Back prop
      optimizer.zero_grad()
      loss.backward()
      loss_train += loss.item()

      #Optimizer
      optimizer.step()

    train_acc = chk_accuracy(train_loader, model)
    val_acc = chk_accuracy(test_loader, model)
    avg_loss = loss_train/(len(train_loader))
    end = time.process_time()

    print('Epoch ({}/{}),Training loss : {:.4f}, Time: {:.2f}, train_accuracy:{:.4f}, val_accuracy:{:.4f}'.format(epoch+1, num_epochs, avg_loss, end - start, train_acc, val_acc))
                                

Epoch (1/100),Training loss : 0.6907, Time: 2.79, train_accuracy:0.9320, val_accuracy:0.9261
Epoch (2/100),Training loss : 0.2283, Time: 3.02, train_accuracy:0.9492, val_accuracy:0.9379
Epoch (3/100),Training loss : 0.1843, Time: 3.03, train_accuracy:0.9506, val_accuracy:0.9412
Epoch (4/100),Training loss : 0.1694, Time: 3.00, train_accuracy:0.9510, val_accuracy:0.9391
Epoch (5/100),Training loss : 0.1651, Time: 3.03, train_accuracy:0.9555, val_accuracy:0.9403
Epoch (6/100),Training loss : 0.1533, Time: 3.07, train_accuracy:0.9628, val_accuracy:0.9485
Epoch (7/100),Training loss : 0.1483, Time: 3.04, train_accuracy:0.9581, val_accuracy:0.9420
Epoch (8/100),Training loss : 0.1449, Time: 3.06, train_accuracy:0.9629, val_accuracy:0.9465
Epoch (9/100),Training loss : 0.1437, Time: 3.10, train_accuracy:0.9655, val_accuracy:0.9462
Epoch (10/100),Training loss : 0.1380, Time: 3.04, train_accuracy:0.9615, val_accuracy:0.9455
Epoch (11/100),Training loss : 0.1350, Time: 3.07, train_accuracy:0.9